<a href="https://colab.research.google.com/github/raxstar5/NLP/blob/main/IMDPsentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [11]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [17]:
dataset = pd.read_csv('/IMDB Dataset.csv')

In [18]:
dataset.shape

(50000, 2)

In [23]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [24]:
nlp = English()
stopwords = list(STOP_WORDS)
punctuations = string.punctuation

In [26]:
def tokenizer(sentence):
    mytokens = nlp(sentence)
    mytokens = [word.lemma_.lower().strip() if word.lemma != "-PRON-" else word.lower_ for word in mytokens]
    mytokens = [word for word in mytokens if word not in stopwords and word not in punctuations]
    return mytokens

In [28]:
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y, **fit_params):
        return self
    def get_params(self, deep = True):
        return {}

def clean_text(text):
    return text.strip().lower()

In [29]:
vectorizer = CountVectorizer(tokenizer = tokenizer, ngram_range=(1,1))
tfvectorizer = TfidfVectorizer(tokenizer = tokenizer)


In [34]:
X = dataset["review"]
y = dataset["sentiment"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 42)


In [35]:
SVCclassifier = LinearSVC()
SVCmodel = Pipeline([("cleaner", predictors()),
                      ('vectorizer', vectorizer),
                       ('classifier',SVCclassifier)])

In [36]:
SVCmodel.fit(X_train,y_train)
SVCpred = SVCmodel.predict(X_test)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [37]:
confusion_matrix(y_test, SVCpred)

array([[4293,  668],
       [ 669, 4370]])

In [40]:
accuracy_score(y_test, SVCpred)*100

86.63

In [41]:
print(classification_report(y_test, SVCpred))

              precision    recall  f1-score   support

    negative       0.87      0.87      0.87      4961
    positive       0.87      0.87      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



In [47]:
pre = SVCmodel.predict(["movie tells what truly means to an artistic brilliance"])
print(pre[0])

positive


In [51]:
pre = SVCmodel.predict(["movie will make you leave the interval way before, i want to claim a refund asap"])
print(pre)

['negative']
